In [ ]:
yoi = [2020]
toi = ['15RTN', '15RTP', '14RNS']
root_path = "C:/Users/mcecil/CGA/CDL/"
spath = root_path + f"CDL_HLS_dataframe{yoi[0]}.csv"
url_file = root_path + f'CDL_HLS_dataframe{yoi[0]}.csv'
index_file = root_path + f'mask_index_{yoi[0]}.csv'
kml_file = root_path + 'sentinel_tile_grid.kml'
geojson_file = root_path + 'aoi.geojson'  ## chip file, lat-long
geojson_rpj_file = root_path + 'aoi_rpj.geojson'
hdf_dir = root_path + 'hdf/'
tiff_dir = root_path + 'tif/'
mask_dir = root_path + 'mask/'

tile_src_path = root_path + "tile_src.csv"

In [ ]:

# Switch plotting backend
import matplotlib
matplotlib.use('agg')

# Import required modules
#import cartopy.crs as ccrs
import os
from datetime import datetime
import pandas as pd
import geopandas
#import pyhdf
import rasterio
import rasterio.mask
import matplotlib.pyplot as pp
import nasa_hls
import numpy as np
import time
import fiona
from pathlib import Path
import time
import urllib.request as urlreq
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:
# Load the HLS tiles and place there coordinates into a numpy array for processing later
fiona.drvsupport.supported_drivers['KML'] = 'rw'
tile_src = geopandas.read_file(kml_file, driver='KML')
tile_name = []
tile_x = []
tile_y = []
for tile_ind in range(tile_src.shape[0]):
    tile_name.append(tile_src.iloc[tile_ind].Name)
    tile_x.append(tile_src.iloc[tile_ind].geometry.centroid.x)
    tile_y.append(tile_src.iloc[tile_ind].geometry.centroid.y)
tile_name = np.array(tile_name)
tile_x = np.array(tile_x)
tile_y = np.array(tile_y)
tile_src = pd.concat([tile_src, tile_src.bounds], axis = 1)
#del tile_src
tile_src.head(5)

,Name,Description,geometry,minx,miny,maxx,maxy
0,01CCV,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -7...,-180.0,-73.064633,180.0,-72.012478
1,01CDH,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-83.835334,180.0,-82.796720
2,01CDJ,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.939452,180.0,-81.906947
3,01CDK,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.044055,180.0,-81.016439
4,01CDL,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-81.148070,180.0,-80.124456


In [4]:
# Load the HLS query csv for this year
qfn = open(url_file)
qtile = []
qyear = []
qmonth = []
qday = []
qdate = []
qurl = []
for i, line in enumerate(qfn):
    if (i == 0): # Skip header
        continue
    dummy = line.split(",")
    qtile.append(dummy[2])
    qdate.append(dummy[3])
    qurl.append(dummy[4])
    
    qday.append(int(dummy[3].split("-")[2]))
    qmonth.append(int(dummy[3].split("-")[1]))
    qyear.append(int(dummy[3].split("-")[0]))
qfn.close()
qdate = np.array(qdate)
qtile = np.array(qtile)
qurl = np.array(qurl)
qmonth = np.array(qmonth)
qyear = np.array(qyear)

In [5]:
# Create the index file
# fn_index = open(index_file, "w")
# fn_index.write('EventID,Name,FireDate,MaskDate,BurnAcres,Tile,MaskFile')
# fn_index.close()

Load geojson used for query. 

In [6]:
aoi_src = geopandas.read_file(geojson_file) ## this could be a list
nfeatures = aoi_src.shape[0]
aoi_src = pd.concat([aoi_src, aoi_src.bounds], axis = 1)
aoi_src['centroid_x'] = aoi_src.centroid.x
aoi_src['centroid_y'] = aoi_src.centroid.y
aoi_src.head(5)

C:\Users\mcecil\AppData\Local\Temp\ipykernel_5544\513036168.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_src['centroid_x'] = aoi_src.centroid.x
C:\Users\mcecil\AppData\Local\Temp\ipykernel_5544\513036168.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_src['centroid_y'] = aoi_src.centroid.y


,geometry,minx,miny,maxx,maxy,centroid_x,centroid_y
0,"POLYGON ((-98.46590 28.48216, -98.46590 28.491...",-98.492844,28.482162,-98.465904,28.491892,-98.479374,28.487027


Loop through aoi_src (chips), associate closest tile, add ID field, download hdf (WORKING)

In [ ]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    aoi_src.at[aoi_ind, 'tile'] = tname
    aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-bs_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):     
        local_name = cu.split('/')[-1].replace("\n", "")
        # Download a candidate file to check cloud cover
        try:
            urlreq.urlretrieve(cu, filename=hdf_dir+local_name)
        except:
            continue
#aoi_src.head(5)

Extract cloud cover and spatial coverage from metadata (NOT WORKING)

In [ ]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    aoi_src.at[aoi_ind, 'tile'] = tname
    aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-bs_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):
       # print(cu)
        local_name = cu.split('/')[-1].replace("\n", "")
        
        if local_name != "HLS.S30.T14RNS.2020117.v1.4.hdf":
            continue
        # Download a candidate file to check cloud cover
        try:
            md = nasa_hls.get_metadata_from_hdf(hdf_dir+local_name, fields=['cloud_cover', 'spatial_coverage'])
            print(md)
       #     if ((md["cloud_cover"] < cldfrac_threshold) and (md['spatial_coverage'] > spacecov_threshold)):
        except:
            continue
        
aoi_src.head(5)

Convert HDF to single-layer TIF (Using HLS library, DOES NOT WORK)

In [ ]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    aoi_src.at[aoi_ind, 'tile'] = tname
    aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-bs_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):
        print(cu)
        local_name = cu.split('/')[-1].replace("\n", "")
        
        if local_name != "HLS.S30.T14RNS.2020117.v1.4.hdf":
            continue
        
        nasa_hls.convert_hdf2tiffs(Path(hdf_dir+local_name), Path(tiff_dir))

Convert HDF to single-layer TIF (Mike's way - works)

In [9]:
import osgeo
from osgeo import gdal
import glob
from subprocess import Popen, PIPE

bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12', 'QA']


hdf_files = glob.glob(hdf_dir + "*.hdf")
for hdf_file in hdf_files:
    print(hdf_file)
    file_name = hdf_file.split('\\')[-1].replace('.hdf', '')
    if file_name != "HLS.S30.T14RNS.2020117.v1.4":
        continue
    print('valid')
    for long_band_name in bands:
        cmd = f'gdal_translate HDF4_EOS:EOS_GRID:"' + hdf_file + '":Grid:' + long_band_name + ' ' + tiff_dir + file_name + '__' + long_band_name + '.tif'
      #  print(cmd)
        p = Popen(cmd, stdout=PIPE, shell=True)
       # print(p)
        output, err = p.communicate()
        #print(output)
        #print(err)
  #  nasa_hls.convert_hdf2tif_dirfs(Path(hdf_file), Path(tif_dir_dir))
    # ds = gdal.Open(hdf_file)
    # ds_t = gdal.Translate('output.tif', hdf_file)
#ds = None
#print(ds)


C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020002.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020005.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020007.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020010.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020012.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020015.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020017.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020020.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020022.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020027.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020032.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020035.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020040.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020042.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020045.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020047.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14R